In [1]:
import requests
import datetime
import base64
import json
import pandas as pd
from urllib.parse import urlencode

In [2]:
client_id = '319b6a63249d40e6850854000f19d265'
client_secret = '7b568c81329a408eaa7db420be7b9047'
redirect_uri = 'https://www.linkedin.com/in/chan-park-036/'
#redirect_uri = 'https%3A%2F%2Fwww.linkedin.com%2Fin%2Fchan-park-036%2F'

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    redirect_uri = ""
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, redirect_uri, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.redirect_uri = redirect_uri
    
    def request_user_auth(self):
        #url = f"https://accounts.spotify.com/authorize?client_id={client_id}&response_type=code&redirect_uri={redirect_uri}&scope=playlist-modify-public%20playlist-modify-private"
        #https://accounts.spotify.com/authorize?client_id=319b6a63249d40e6850854000f19d265&response_type=code&redirect_uri=https://www.linkedin.com/in/chan-park-036/&scope=playlist-modify-public%20playlist-modify-private
        auth_url = 'https://accounts.spotify.com/authorize'
        auth_code = requests.get(auth_url, {
            'client_id': self.client_id,
            'redirect_uri': self.redirect_uri,
            'response_type': "code",
            'scope': 'playlist-modify-private playlist-modify-public',
        })
        print(auth_code.url)
        return 'AQBSNVSIgLkrgmTF7199ljyriGTse3ouDe7sb3JXJmZqWOLJc_lUv8WNZA-9qkVNNkS0dE74EI_UMn6fM5tvM7lEgBRI-yQSYfomf_aFalLspOsAOunapxx1xrGIis8yhGRH-iJELTO--xopYZIObH8Rzxqui-H2CAlGtyg8hZf0Nks87RS0FCKSjUb1Ufz7l_xnulpLt3Bmi9Aykbp4f7Re9zw7FlX2FjsFlLWtH1IOOdQNfBTwlq88RgW9ekQ'
        
        
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("set client id and secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
    
    def get_token_header(self):
        return {
            "Authorization": f"Basic {self.get_client_credentials()}",
            "Content-Type":'application/x-www-form-urlencoded'
        }
        
    def get_token_data(self):
        return {
            "grant_type": "authorization_code",
            "code": f"{self.request_user_auth()}",
            "redirect_uri": f"{self.redirect_uri}",
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        print(token_data)
        token_header = self.get_token_header()
        r = requests.post(token_url, data=token_data, headers=token_header)
        print(r)
        if r.status_code not in range(200,299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data["access_token"]
        expires_in = data["expires_in"]
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
        

In [4]:
#check authorization code
client = SpotifyAPI(client_id,client_secret, redirect_uri)
client.request_user_auth()

https://accounts.spotify.com/en/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Dplaylist-modify-private%2Bplaylist-modify-public%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fwww.linkedin.com%252Fin%252Fchan-park-036%252F%26client_id%3D319b6a63249d40e6850854000f19d265


'AQBSNVSIgLkrgmTF7199ljyriGTse3ouDe7sb3JXJmZqWOLJc_lUv8WNZA-9qkVNNkS0dE74EI_UMn6fM5tvM7lEgBRI-yQSYfomf_aFalLspOsAOunapxx1xrGIis8yhGRH-iJELTO--xopYZIObH8Rzxqui-H2CAlGtyg8hZf0Nks87RS0FCKSjUb1Ufz7l_xnulpLt3Bmi9Aykbp4f7Re9zw7FlX2FjsFlLWtH1IOOdQNfBTwlq88RgW9ekQ'

In [5]:
#perform authentication and get token
client = SpotifyAPI(client_id,client_secret, redirect_uri)
client.perform_auth()
access_token = client.access_token

https://accounts.spotify.com/en/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Dplaylist-modify-private%2Bplaylist-modify-public%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fwww.linkedin.com%252Fin%252Fchan-park-036%252F%26client_id%3D319b6a63249d40e6850854000f19d265
{'grant_type': 'authorization_code', 'code': 'AQBSNVSIgLkrgmTF7199ljyriGTse3ouDe7sb3JXJmZqWOLJc_lUv8WNZA-9qkVNNkS0dE74EI_UMn6fM5tvM7lEgBRI-yQSYfomf_aFalLspOsAOunapxx1xrGIis8yhGRH-iJELTO--xopYZIObH8Rzxqui-H2CAlGtyg8hZf0Nks87RS0FCKSjUb1Ufz7l_xnulpLt3Bmi9Aykbp4f7Re9zw7FlX2FjsFlLWtH1IOOdQNfBTwlq88RgW9ekQ', 'redirect_uri': 'https://www.linkedin.com/in/chan-park-036/'}
<Response [200]>


In [6]:
#search example
headers = {
    "Authorization": f"Bearer {access_token}"
}
search_endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q":"Entropy", "type": "track"})
lookup_url = f"{search_endpoint}?{data}"
r = requests.get(lookup_url,headers=headers)


In [7]:
#get spotify recommendations
rec_endpoint = "https://api.spotify.com/v1/recommendations?"

limit=10
market="US"
seed_genres="indie"
target_danceability=0.9
query = f'{rec_endpoint}limit={limit}&market={market}&seed_genres={seed_genres}&target_danceability={target_danceability}'
response =requests.get(query, 
               headers={"Content-Type":"application/json", 
                        "Authorization": f"Bearer {access_token}"})
json_response = response.json()
uris = []
for i in json_response['tracks']:
            uris.append(i['uri'])
            print(f"\"{i['name']}\" by {i['artists'][0]['name']}")


"Rawnald Gregory Erickson the Second" by STRFKR
"Fidelity" by Regina Spektor
"Where I'm From" by Digable Planets
"Summer Days" by Rhye
"Wraith Pinned to the Mist and Other Games" by of Montreal
"Pretty Girl" by Clairo
"This Must Be the Place (Naive Melody) - 2005 Remaster" by Talking Heads
"FACE" by BROCKHAMPTON
"anemone" by slenderbodies
"South of the River" by Tom Misch


In [8]:
#find my 10 most recently created playlists

limit = 10
offset = 0
user_id = "e0fu1tvd4el2atkqmvlc2m6br"
curr_user_playlist_endpoint = f"https://api.spotify.com/v1/users/{user_id}/playlists?"
query = f'{curr_user_playlist_endpoint}user_id={user_id}'
response =requests.get(query, 
               headers={"Content-Type":"application/json", 
                        "Authorization": f"Bearer {access_token}"})
json_response = response.json()
playlist_ids = []
for i in json_response["items"]:
        print(f"\"{i['name']}\" and {i['id']}")
        playlist_ids.append(i['id'])

"dumbertime" and 0g1jZVZ8VNcxeF7g0yjFn8
"neverending" and 2fRFWEkmtpMfvjqMvkFrdc
"painkillers for work" and 7x1ZKrx9eJ4VGrj7JN9eIs
"covid therapy" and 7C64t0Ms6Vm0Z9xRUieTbe
"late" and 79IZw6BE8CnGLj345QUsaw
"chill k" and 0DE2Bl1yjFgtcJ7csm0DPV
"초롱" and 6Rlgzs2NSzL0QdqjViE4Mg
"still" and 5VzN0DravpnKrHu8s6AAUa
"soft" and 2Fp1SgXkBKC6jb6z9TA4Od
"chill" and 26IeIVee8N4KBJvtA1rV88


In [ ]:
#get items from 4 most recent playlists
playlist_ids = playlist_ids[:4]
print(playlist_ids)
p_ids = []
p_dic = {}
for playlist_id in playlist_ids:
    get_playlist_item_endpoint = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    query = f'{get_playlist_item_endpoint}'
    response =requests.get(query, 
                   headers={"Content-Type":"application/json", 
                            "Authorization": f"Bearer {access_token}"})
    #print(response.json())
    for i in response.json()["items"]:
        p_dic[i['track']['id']] = i['track']['uri']
p_dic_list = list(p_dic.keys())


    

In [ ]:
#create a dataframe using each track's id's and uri's
p_count_l = 1
p_count_r = 1

#initial df
get_track_info_endpoint = f"https://api.spotify.com/v1/audio-features?ids={p_dic_list[0]}"
response =requests.get(get_track_info_endpoint, 
                   headers={"Content-Type":"application/json", 
                            "Authorization": f"Bearer {access_token}"})
df = pd.DataFrame(response.json()['audio_features'],columns=('id','uri','acousticness','danceability',\
                                                             'energy','instrumentalness','duration_ms','key','liveness','loudness','mode',\
                                                             'speechiness','tempo','time_signature'))
#append by every 99
while p_count_r <= len(p_dic_list):
    p_count_r += 99
    x = ",".join(p_dic_list[p_count_l:p_count_r])
    get_track_info_endpoint = f"https://api.spotify.com/v1/audio-features?ids={x}"
    response =requests.get(get_track_info_endpoint, 
                   headers={"Content-Type":"application/json", 
                            "Authorization": f"Bearer {access_token}"})
    df = df.append(pd.DataFrame(response.json()['audio_features'],columns=('id','uri','acousticness','danceability',\
                                                             'energy','instrumentalness','duration_ms','key','liveness','loudness','mode',\
                                                             'speechiness','tempo','time_signature')))
    p_count_l += 99
    

df = df.reset_index(drop=True)
    
print(df)
    #for i in response.json():
            #acoustics.append(i)
    
    #dflist.append([track_id,track_uri,acoustics])
#df = pd.DataFrame(dflist)
#print(df)

In [ ]:
#create 3 different dataframes based on certain conditions among the songs in the dataframe

#drive 
df_drive = df[(df['tempo'] >= 60) & (df['tempo'] < 90)]
print(len(df_drive))

# chill
df_chill = df[(df['tempo'] >= 90) & (df['tempo'] <= 130) & (df['energy'] >= 0.3) & (df['energy'] <= 0.6)]
print(len(df_chill))

# hype
df_hype = df[(df['tempo'] > 130) & (df['danceability'] >= 0.35) & (df['danceability'] <= 0.8)]
print(len(df_hype))

playlist_descriptors = [('drive','for driving at night'), ('chill','study hard >:)'), ('hype','work out grind')]
playlist_dfs = [df_drive,df_chill,df_hype]

In [ ]:
#create 3 different playlists using the 3 different descriptors

user_id = "e0fu1tvd4el2atkqmvlc2m6br"
create_playlist_endpoint = f"https://api.spotify.com/v1/users/{user_id}/playlists"
c_playlist_ids = {}
playlist_counter = 0
for name,desc in playlist_descriptors:
    request_body = json.dumps({
              "name": name,
              "description": desc,
              "public": False 
            })
    response = requests.post(url = create_playlist_endpoint, data = request_body, headers={"Content-Type":"application/json", 
                            "Authorization": f"Bearer {access_token}"})
    c_playlist_ids[response.json()['id']] = playlist_dfs[playlist_counter]
    playlist_counter +=1


In [ ]:
print(c_playlist_ids.keys())

In [ ]:
#load playlists with the 3 different dataframes
for c_playlist_id in c_playlist_ids:
    fill_playlist_endpoint = f"https://api.spotify.com/v1/playlists/{c_playlist_id}/tracks"
    request_body = json.dumps({
              "uris" : c_playlist_ids[c_playlist_id]['uri'].tolist()
            })
    response = requests.post(url = fill_playlist_endpoint, data = request_body, headers={"Content-Type":"application/json", 
                            "Authorization": f"Bearer {access_token}"})
